# S3. Fine-tuning de modelo pre-entrenado InceptionResNetV2 para la clasificación de CIFAR-10
Este cuaderno realiza un fine-tuning del [modelo InceptionResNetV2](https://keras.io/api/applications/inceptionresnetv2/) pre-entrenado con la base de datos de imágenes [ImageNet](https://www.image-net.org) para la clasificación de CIFAR-10. 

### Carga de datos
La utilización de una red pre-entrenada conlleva preprocesar nuestros datos con el mismo preproceso que se empleó para los datos de entrenamiento de la red pre-entrenada. En el caso de la red Inception V3, las imágenes necesitan, entre otras cosas, ser redimensionadas a 299 x 299. Si realizamos este preproceso para nuestro conjunto de entrenamiento, esto requeriría aproximadamente 40GB, por lo que es conveniente realizar este preproceso bajo demanda. Es decir, aplicaremos el preproceso para cada batch en el momento que vaya a ser utilizado.      

La aplicación de este preproceso bajo demanda está ya implementado en el módulo [tensorflow_datasets](https://www.tensorflow.org/datasets), así que en esta sesión cargaremos y manipularemos CIFAR-10 como un objeto [Dataset](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) utilizando este módulo.

In [12]:
import tensorflow as tf
import tensorflow_datasets as tfds
train_data, test_data = tfds.load('cifar10', split=['train', 'test'], as_supervised=True)
train_size = len(train_data)

### Preproceso

Primero definimos una función que aplica el preproceso necesario a una muestra de entrenamiento (imagen, etiqueta de clase). Esto incluye redimensionar la imagen a 299 x 299, el preproceso específico de la red InceptionV3 y convertir la etiqueta de clase a one-hot encoding.

In [13]:
from keras.applications.inception_resnet_v2 import preprocess_input

img_size = (299, 299)
num_classes = 10

def preprocess(image, label):
    image = tf.image.resize(image, img_size)
    image = tf.cast(image, tf.float32)
    image = preprocess_input(image)
    label = tf.one_hot(label, num_classes)
    return image, label

A continuación indicamos que la función anteriormente definida se aplicará a cada imagen cuando sea necesario. Para ello utilizamos la función [map()](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#map) del objeto [Dataset](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) y le pasamos como parámetro la función de preproceso que queremos que sea aplique a cada muestra del conjunto de datos.

In [14]:
train_data = train_data.map(preprocess)
test_data = test_data.map(preprocess)

Las funciones [take()](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#take) y [skip()](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#skip) combinadas permiten definir los conjuntos de entrenamiento y validación como nuevos Datasets. 

In [15]:
train_size = int(0.8 * train_size)
train_dataset = train_data.take(train_size)
val_dataset = train_data.skip(train_size)
test_dataset = test_data

print(len(train_dataset),len(val_dataset))

40000 10000


### Carga del modelo pre-entrenado

Seguidamente procedemos con la carga del modelo Inception V3 con los pesos resultantes de entrenarlo con la base de datos Imagenet, pero no queremos que el modelo incluya la capa de salida (include_top=False) que por defecto es una softmax de 1000 clases.

In [3]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2

model = InceptionResNetV2(input_shape=img_size + (3,),include_top=False, weights='imagenet')
model.summary()

219055592/219055592 [==============================] - 338s 2us/step
Model: "inception_resnet_v2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 299, 299, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 149, 149, 32)         864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 149, 149, 32)         96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation

### Preparación del modelo pre-entrenado

Vamos a preparar la red Inception V3 para ser entrenada (fine-tuning) con CIFAR-10. Dado el número de parámetros de este modelo (21M), nos limitaremos a utilizarlo con los valores por defecto y añadiremos una capa GlobalAveragePooling + MLP seguida de una softmax de 10 neuronas (10 clases) acorde a CIFAR-10 que sí que entrenaremos.

In [4]:
from keras.layers import GlobalAveragePooling2D, Dense, Dropout
from keras.models import Model

for layer in model.layers:
    layer.trainable = False

x = GlobalAveragePooling2D()(model.output)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=model.input, outputs=output)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 299, 299, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 149, 149, 32)         864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 149, 149, 32)         96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 149, 149, 32)         0         ['batch_normalization[0][0

Compilamos el modelo con los mismos parámetros que en sesiones anteriores.

In [5]:
from keras.optimizers import Adam

opt=Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

Entrenamos el modelo utilizando los conjuntos de datos organizado en batches y que son cargados en memoria dinámicamente.

In [16]:
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.models import load_model

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.00001)
checkpoint = ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

epochs=10
batch_size=32
train_dataset_batched = train_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
val_dataset_batched = val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
history = model.fit(train_dataset_batched,
                    epochs=epochs,
                    verbose=1,
                    validation_data=val_dataset_batched,
                    callbacks=[reduce_lr,checkpoint])

Epoch 1/10


2023-12-09 11:11:21.402129: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8906
2023-12-09 11:11:21.453250: W external/local_xla/xla/stream_executor/gpu/asm_compiler.cc:225] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.9
2023-12-09 11:11:21.453265: W external/local_xla/xla/stream_executor/gpu/asm_compiler.cc:228] Used ptxas at ptxas
2023-12-09 11:11:21.453302: W external/local_xla/xla/stream_executor/gpu/redzone_allocator.cc:322] UNIMPLEMENTED: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2023-12-09 11:11:25.524817: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2023-12-09 11:11:27.349495: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kerne

1250/1250 [==============================] - ETA: 0s - loss: 0.4452 - accuracy: 0.8561

2023-12-09 11:15:30.216502: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.



Epoch 1: val_accuracy improved from -inf to 0.89370, saving model to best_model.h5


/home/jorcisai/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1250/1250 [==============================] - 322s 248ms/step - loss: 0.4452 - accuracy: 0.8561 - val_loss: 0.3189 - val_accuracy: 0.8937 - lr: 0.0010
Epoch 2/10
1250/1250 [==============================] - ETA: 0s - loss: 0.3506 - accuracy: 0.8848
Epoch 2: val_accuracy improved from 0.89370 to 0.90590, saving model to best_model.h5
1250/1250 [==============================] - 304s 244ms/step - loss: 0.3506 - accuracy: 0.8848 - val_loss: 0.2840 - val_accuracy: 0.9059 - lr: 0.0010
Epoch 3/10
1250/1250 [==============================] - ETA: 0s - loss: 0.3270 - accuracy: 0.8901
Epoch 3: val_accuracy improved from 0.90590 to 0.90730, saving model to best_model.h5
1250/1250 [==============================] - 304s 243ms/step - loss: 0.3270 - accuracy: 0.8901 - val_loss: 0.2777 - val_accuracy: 0.9073 - lr: 0.0010
Epoch 4/10
1250/1250 [==============================] - ETA: 0s - loss: 0.3107 - accuracy: 0.8973
Epoch 4: val_accuracy did not improve from 0.90730
1250/1250 [======================

### Cargar el mejor modelo y evaluarlo con el test set

In [17]:
model = load_model('best_model.h5')
test_dataset_batched = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
score = model.evaluate(test_dataset_batched, verbose=0)
print(f'Test loss: {score[0]*100:.2f}')
print(f'Test accuracy: {score[1]*100:.2f}')

Test loss: 25.86
Test accuracy: 91.77


### Aumento de datos

En las sesiones anteriores, hemos utilizado la función [ImageDataGenerator](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator) para realizar el aumento de datos. Sin embargo, esta función no se recomienda para los nuevos desarrollos de código por estar obsoleta, y en su lugar se deben utilizar las [capas de preproceso](https://www.tensorflow.org/guide/keras/preprocessing_layers). Más concretamente, utilizaremos algunas de las [capas de preproceso de aumento de datos para imágenes](https://www.tensorflow.org/guide/keras/preprocessing_layers#image_data_augmentation).

Añadiremos estas capas de aumento de datos antes del modelo Inception V3.

In [18]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout, RandomRotation, RandomTranslation, RandomZoom
from keras.models import Model

input_layer = Input(shape=img_size + (3,))

x = RandomRotation(factor=0.1, fill_mode='nearest')(input_layer)
x = RandomTranslation(height_factor=0.1, width_factor=0.1, fill_mode='nearest')(x)
x = RandomZoom(height_factor=0.2, fill_mode='nearest')(x)

inception_model = InceptionResNetV2(input_shape=img_size + (3,),include_top=False, weights='imagenet')

for layer in inception_model.layers:
    layer.trainable = False

x = inception_model(x)

x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(num_classes, activation='softmax')(x)

aug_model = Model(inputs=input_layer, outputs=output)

aug_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 299, 299, 3)]     0         
                                                                 
 random_rotation (RandomRot  (None, 299, 299, 3)       0         
 ation)                                                          
                                                                 
 random_translation (Random  (None, 299, 299, 3)       0         
 Translation)                                                    
                                                                 
 random_zoom (RandomZoom)    (None, 299, 299, 3)       0         
                                                                 
 inception_resnet_v2 (Funct  (None, 8, 8, 1536)        54336736  
 ional)                                                          
                                                           

In [19]:
from keras.optimizers import Adam

opt=Adam(learning_rate=0.001)
aug_model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

In [20]:
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.models import load_model

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.00001)
checkpoint = ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

epochs=20
batch_size=32
train_dataset_batched = train_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
val_dataset_batched = val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
history = aug_model.fit(train_dataset_batched,
                    epochs=epochs,
                    verbose=1,
                    validation_data=val_dataset_batched,
                    callbacks=[reduce_lr,checkpoint])

Epoch 1/20


2023-12-09 13:11:38.513772: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.


1250/1250 [==============================] - ETA: 0s - loss: 0.7988 - accuracy: 0.7282
Epoch 1: val_accuracy improved from -inf to 0.88330, saving model to best_model.h5


/home/jorcisai/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1250/1250 [==============================] - 317s 249ms/step - loss: 0.7988 - accuracy: 0.7282 - val_loss: 0.3452 - val_accuracy: 0.8833 - lr: 0.0010
Epoch 2/20
1250/1250 [==============================] - ETA: 0s - loss: 0.6816 - accuracy: 0.7674
Epoch 2: val_accuracy improved from 0.88330 to 0.89410, saving model to best_model.h5
1250/1250 [==============================] - 308s 247ms/step - loss: 0.6816 - accuracy: 0.7674 - val_loss: 0.3238 - val_accuracy: 0.8941 - lr: 0.0010
Epoch 3/20
1250/1250 [==============================] - ETA: 0s - loss: 0.6502 - accuracy: 0.7777
Epoch 3: val_accuracy improved from 0.89410 to 0.89970, saving model to best_model.h5
1250/1250 [==============================] - 308s 246ms/step - loss: 0.6502 - accuracy: 0.7777 - val_loss: 0.3055 - val_accuracy: 0.8997 - lr: 0.0010
Epoch 4/20
1250/1250 [==============================] - ETA: 0s - loss: 0.6346 - accuracy: 0.7829
Epoch 4: val_accuracy did not improve from 0.89970
1250/1250 [======================

In [21]:
aug_model = load_model('best_model.h5')
test_dataset_batched = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
score = aug_model.evaluate(test_dataset_batched, verbose=0)
print(f'Test loss: {score[0]*100:.2f}')
print(f'Test accuracy: {score[1]*100:.2f}')

Test loss: 25.66
Test accuracy: 91.18
